In [ ]:
# Import libraries
import time
import random
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import datetime

# Download the Vader lexicon
nltk.download('vader_lexicon')

# Parameters
n = 3
tickers = ['AAPL', 'TSLA', 'AMZN', 'MSFT', 'META', 'NVDA', 'GOOGL']

# Get Data
finwiz_url = 'https://finviz.com/quote.ashx?t='
news_tables = {}

for ticker in tickers:
    url = finwiz_url + ticker
    req = Request(url=url, headers={'User-Agent': 'Mozilla/5.0'})
    resp = urlopen(req)
    html = BeautifulSoup(resp, features="lxml")
    news_table = html.find(id='news-table')
    news_tables[ticker] = news_table
    time.sleep(random.randint(1, 3))  # Ajout d'un délai aléatoire entre les requêtes pour éviter le blocage

try:
    for ticker in tickers:
        df = news_tables[ticker]
        df_tr = df.findAll('tr')

        print ('\n')
        print ('Recent News Headlines for {}: '.format(ticker))

        for i, table_row in enumerate(df_tr):
            a_text = table_row.a.text
            td_text = table_row.td.text
            td_text = td_text.strip()
            print(a_text,'(',td_text,')')
            if i == n-1:
                break
except KeyError:
    pass

# Iterate through the news
parsed_news = []
for file_name, news_table in news_tables.items():
    for x in news_table.findAll('tr'):
        text = x.a.get_text()
        date_scrape = x.td.text.split()

        if len(date_scrape) == 1:
            time = date_scrape[0]
            if time == 'Today':
                time = datetime.date.today().strftime('%Y-%m-%d')  # Format the date as YYYY-MM-DD

        else:
            date = date_scrape[0]
            time = date_scrape[1]

        ticker = file_name.split('_')[0]

        parsed_news.append([ticker, date, time, text])

# Sentiment Analysis
analyzer = SentimentIntensityAnalyzer()

columns = ['Ticker', 'Date', 'Time', 'Headline']
news = pd.DataFrame(parsed_news, columns=columns)
scores = news['Headline'].apply(analyzer.polarity_scores).tolist()

df_scores = pd.DataFrame(scores)
news = news.join(df_scores, rsuffix='_right')


# View Data
news['Date'] = pd.to_datetime(news.Date, errors='coerce')

unique_ticker = news['Ticker'].unique().tolist()
news_dict = {name: news.loc[news['Ticker'] == name] for name in unique_ticker}

values = []
for ticker in tickers:
    dataframe = news_dict[ticker]
    dataframe = dataframe.set_index('Ticker')
    dataframe = dataframe.drop(columns=['Headline'])
    print ('\n')
    print (dataframe.head())

    mean = round(dataframe['compound'].mean(), 2)
    values.append(mean)

df = pd.DataFrame(list(zip(tickers, values)), columns=['Ticker', 'Mean Sentiment'])
df = df.set_index('Ticker')
df = df.sort_values('Mean Sentiment', ascending=False)
print ('\n')
print (df)


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!




Recent News Headlines for AAPL: 
Warren Buffett Just Revealed the 8 Stocks That Berkshire Hathaway Will Likely Hold Forever -- and Apple Wasn't One of Them ( Today 06:07AM )
Apple and Tesla veterans aim to help Japan design AI chips ( 04:35AM )
Samsung's XR device expected to enter production before end of 2024 ( Feb-26-24 11:00PM )


Recent News Headlines for TSLA: 
Forget Nvidia: These 3 Artificial Intelligence (AI) Stocks Have Up to 203% Upside, According to Select Wall Street Analysts ( Today 05:21AM )
Apple and Tesla veterans aim to help Japan design AI chips ( 04:35AM )
Ford halts shipments of popular EV over quality, casts uncertainty ( Feb-26-24 09:31PM )


Recent News Headlines for AMZN: 
Walmart Stock Has 30% Upside, According to 1 Wall Street Analyst ( Today 05:51AM )
Numerator Retailer Snapshots: Most U.S. households shop Amazon ( Feb-26-24 06:10PM )
Berkshire Hathaway Doesn't Hold Nvidia Stock Will Warren Buffett Come to Regret it? ( 06:00PM )


Recent News Headlines for